# CSE391
## Data Science
### Final exam



You are given 70,000 images (each image consists of 28*28 pixels) of hand-written digits from 0-9.
The task is to design a classifier that can recognize these images. In other words, you input an image from this dataset and the output is a pridected number from 0 to 9.

For this problem, we only focus on classification task between two digits: 3, and 8 in one round and 5 and 6 in the other round, meaning that you deal with a binary classification task. 


The dataset contains 60,000 images for training and 10,000 for test. 
Do the following:
1. Extract all images contaning digits 3 and 8 both in training and test data set.
2.	Plot an image of digit 3 from training data. It could be any image of digit 3.
3.	Train a feedforward neural network with the following architecture: 
    - Three layers
    -	256 hidden units
    -	feedforward
    -   number of epoch = 10
4.	Print out the accuracy of your model for each epoch both for the test and train sets
5.	Report the results in terms of a confusion matrix
6.	Repeat step 3,4,and 5 but with 25 hidden units
7.	Repeat  1-6 with input images 5 and 6



In [2]:
import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix


(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print(x_train.shape)
print(x_test.shape)

## extract 3 and 8 from train and test sets
xtrain3 = x_train[ y_train == 3,:,:]
xtrain8 = x_train[ y_train == 8,:,:]
xtest3 = x_test[y_test == 3,:,:]
xtest8 = x_test[y_test == 8,:,:]

ytrain3 = x_train[ y_train == 3,:,:]
ytrain8 = x_train[ y_train == 8,:,:]
ytest3 = x_test[y_test == 3,:,:]
ytest8 = x_test[y_test == 8,:,:]


trainx =  np.concatenate((xtrain3, xtrain8), axis=0)
testx = np.concatenate((xtest3, xtest8), axis=0)
trainy =  np.concatenate((ytrain3, ytrain8), axis=0)
testy = np.concatenate((ytest3, ytest8), axis=0)

print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)


## show an image of 3 from the training data
plt.imshow(xtrain3[0])
plt.show()

(60000, 28, 28)
(10000, 28, 28)
(11982, 28, 28)
(1984, 28, 28)
(11982, 28, 28)
(1984, 28, 28)


<Figure size 640x480 with 1 Axes>

In [ ]:
# Implementation of a simple MLP network with one hidden layer. Tested on the iris data set.
# Requires: numpy, sklearn>=0.18.1, tensorflow>=1.0

# NOTE: In order to make the code simple, we rewrite x * W_1 + b_1 = x' * W_1'
# where x' = [x | 1] and W_1' is the matrix W_1 appended with a new row with elements b_1's.
# Similarly, for h * W_2 + b_2

# author :vinhkhuc  Feb 26, 2017
import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)


def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

# def get_iris_data():
#     """ Read the iris data set and split them into training and test sets """
#     iris   = datasets.load_iris()
#     data   = iris["data"]
#     target = iris["target"]
#     print(target)
#     # Prepend the column of 1s for bias
#     N, M  = data.shape
#     all_X = np.ones((N, M + 1))
#     all_X[:, 1:] = data

#     # Convert into one-hot vectors
#     num_labels = len(np.unique(target))
#     all_Y = np.eye(num_labels)[target]  # One liner trick!
#     return train_test_split(all_X, all_Y, test_size=0.33, random_state=RANDOM_SEED)

def main():
    
    global trainx
    global testx
    global trainy
    global testy
    
    train_X = trainx/255.0
    test_X = testx/255.0
    train_y = trainy/255.0
    test_y = testy/255.0
    
    train_X = train_X.flatten() 
    test_X = test_X.flatten() 
    train_y = train_y.flatten() 
    test_y = test_y.flatten() 
    
    
    
    
    #train_X, test_X, train_y, test_y = get_iris_data()

    # Layer's sizes
    x_size = train_X.shape[0]   # Number of input nodes: 4 features and 1 bias
    h_size = 256                # Number of hidden nodes
    y_size = train_y.shape[0]   # Number of outcomes (3 iris flowers)

    # Symbols
    X = tf.placeholder("float", shape=[None, x_size])
    y = tf.placeholder("float", shape=[None, y_size])

    # Weight initializations
    w_1 = init_weights((x_size, h_size))
    w_2 = init_weights((h_size, y_size))

    # Forward propagation
    yhat    = forwardprop(X, w_1, w_2)
    predict = tf.argmax(yhat, axis=1)

    # Backward propagation
    cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    updates = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

    # Run SGD
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    for epoch in range(100):
        # Train with each example
        for i in range(len(train_X)):
            sess.run(updates, feed_dict={X: train_X[i: i + 1], y: train_y[i: i + 1]})

        train_accuracy = np.mean(np.argmax(train_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: train_X, y: train_y}))
        test_accuracy  = np.mean(np.argmax(test_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: test_X, y: test_y}))

        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
              % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))

    sess.close()

if __name__ == '__main__':
    main()






